<a href="https://colab.research.google.com/github/Hyorim-Kim/colab/blob/main/tfc41s2s_engkor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# seq2seq를 이용해 글자 단위 번역기 (영어 => 한국어)
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 1024  # 인코딩 공간의 잠재 차원
data_path = 'kor.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding='utf-8').read().split('\n')
print(len(lines))


5893


In [6]:
for line in lines[:len(lines) -1]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'   # tab을 문장의 시작<sos>. enter를 문장의 끝<eos>
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:   # 영어
    if char not in input_characters:
      input_characters.add(char)    # 중복을 배제한 영어 글자 기억

  for char in target_text:   # 영어
    if char not in target_characters:
      target_characters.add(char)    # 중복을 배제한 영어 글자 기억

print(input_texts)
print(target_texts)
print(input_characters)
print(target_characters)

input_characters = sorted(list(input_characters))  # 원문 갱신
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

print('특수문자 포함 영어 관련 글자 수 : ', len(input_characters))  # 74
print('특수문자 포함 영어 관련 글자 : ', input_characters)
print('특수문자 포함 한국어 관련 글자 수 : ', len(target_characters))  # 1010
print('특수문자 포함 한국어 관련 글자 : ', target_characters)

max_encoder_seq_len = max([len(t) for t in input_texts])
max_decoder_seq_len = max([len(t) for t in target_texts])
print('영어 단어 중 가장 긴 단어 글자 수 : ', max_encoder_seq_len)
print('한국어 단어 중 가장 긴 단어 글자 수 : ', max_decoder_seq_len)

# 글자 집합에 글자 단위로 저장된 각 글자에 대해 index를 부여
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
print(input_token_index)  # 영어 {' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7, '.': 8, ...
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(target_token_index)  # 한국어

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
# print(encoder_input_data)
print(encoder_input_data.shape)  # (5892, 537, 74)
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_input_data.shape)  # (5892, 298, 1010)
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_target_data.shape)  # (5892, 298, 1010)

# 0으로 채워진 배열에 해당 글자가 있는 지점에는 1을 기억
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  # print(i, input_text, target_text)  # 11688 If you are going abroad, it's necessary to have a passport. 	외국에 가려면 여권이 필요합니다.
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1  # 해당 글자가 있는 경우 1을 기억
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1  # 해당 글자가 있는 경우 1을 기억
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1  # decoder_target_data가 한 step 앞서 진행됨. 시작문자는  포함되지 않음.
  # print(encoder_input_data)
  # 수정 ***

['Go.', 'Hi.', 'Run!', 'Run.', 'Who?', 'Wow!', 'Duck!', 'Fire!', 'Help!', 'Hide.', 'Jump!', 'Jump.', 'Stay.', 'Wait!', 'Wait!', 'Wait.', 'Begin.', 'Hello!', 'Hello.', 'I see.', 'I try.', 'I won!', 'Oh no!', 'Relax.', 'Shoot!', 'Smile.', 'Sorry?', 'Sorry?', 'Sorry?', 'Attack!', 'Attack!', 'Freeze!', 'Get up.', 'Got it!', 'Got it?', 'He ran.', 'Hug me.', 'I know.', 'I lost.', 'I quit.', 'I work.', 'Listen.', 'No way!', 'No way!', 'Thanks.', 'We try.', 'We won.', 'Why me?', 'Awesome!', 'Be calm.', 'Be calm.', 'Be fair.', 'Beat it.', 'Call us.', 'Come in.', 'Come on!', 'Get out.', 'Go away!', 'Go away.', 'Goodbye!', 'Goodbye.', 'He came.', 'He came.', 'He runs.', 'Help me!', 'Help me.', 'Hit Tom.', 'Hold it!', 'Hold it!', 'Hold it!', 'Hold it!', 'I agree.', "I'm sad.", 'Join us.', 'Kiss me.', 'Kiss me.', 'Me, too.', 'Open up.', 'Perfect!', 'Show me.', 'Shut up!', 'Skip it.', 'Stop it.', 'Tell me.', 'Text me.', 'Tom won.', 'Wake up!', 'Wash up.', 'We know.', 'We lost.', 'Welcome.', 'Welcome

In [8]:
# 입력 시퀀스를 정의 후 처리 : Functional api 사용
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)  # return_sequences=True/False에 상관없이 마지막 은닉상태를 출력함
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# encoder_outputs은 버리고 은닉상태와 셀상태만 유지
encoder_states = [state_h, state_c]  # 이게 바로 context vector(문맥 벡터)
